In [30]:
from datetime import datetime
import re
import json
import csv
from io import StringIO
import csvfile as csvfile
from bs4 import BeautifulSoup
import requests
import pandas

url_stats = 'https://uk.finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://uk.finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://uk.finance.yahoo.com/quote/{}/financials?p={}'
companies = ['TSLA', 'MSFT', 'AAPL', 'AMZN', 'FB', 'GOOG', 'NFLX', 'SPOT', 'ZM', 'DBX', 'KO', 'DIS', 'PEP', 'ADBE', 'SONO', 'AMD', 'NVDA', 'INTC', 'PYPL', 'CRSR']
stock = 'TSLA'

#Statistics
response = requests.get(url_stats.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

now = datetime.today().strftime('%Y-%m-%d')
print(now)

#Historical Stock Data
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/{}?'

params = {
    'start' : now,
    'interval' : '1d',
    'range' : '1y',
    'events' : 'history'
}

# https://query1.finance.yahoo.com/v7/finance/download/TSLA?period1=1576852747&period2=1608475147&interval=1d&events=history&includeAdjustedClose=true

response = requests.get(stock_url.format(stock), params=params)
response.text


for s in companies:
    response = requests.get(stock_url.format(s), params=params)
    response.text

    file = StringIO(response.text)
    reader = csv.DictReader(file)
    with open('./csvs/'+s+'.csv', 'w', newline='') as f:
        fieldnames = ['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']
        writer = csv.DictWriter(f, fieldnames=fieldnames)

        writer.writeheader()
        for row in reader:
            writer.writerow({'Date': row['Date'], 'Open': row['Open'], 'High': row['High'], 'Low': row['Low'], 'Close': row['Close'], 'Adj Close': row['Adj Close'], 'Volume': row['Volume']})

2020-12-20
